# Level 4: Agentic RAG

This tutorial presents an example of executing queries with agentic RAG in Llama Stack. It shows how to initialize an agent with the RAG tool provided by Llama Stack and to invoke it such that retrieval from a vector DB is activated when necessary. The tutorial also covers document ingestion using the RAG tool.
For a foundational (non-agentic) RAG tutorial, please refer to [Level1_foundational_RAG.ipynb](demos/rag_agentic/notebooks/Level1_foundational_RAG.ipynb).

## Overview

This tutorial covers the following steps:
1. Connecting to a llama-stack server.
2. Indexing a collection of documents in a vector DB for later retrieval.
3. Initializing the agent capable of retrieving content from vector DB via tool use.
4. Launching the agent and using it to answer user queries during the inference step.


## Prerequisites

Before starting, ensure you have a running instance of the Llama Stack server (local or remote) with at least one preconfigured vector DB. For more information, please refer to the corresponding [Llama Stack tutorials](https://llama-stack.readthedocs.io/en/latest/getting_started/index.html).

## Setting the Environment Variables

Use the [`.env.example`](../../../.env.example) to create a new file called `.env` and ensure you add all the relevant environment variables below.

In addition to the environment variables listed in the ["Getting Started" notebook](demos/rag_agentic/notebooks/Level0_getting_started_with_Llama_Stack.ipynb), the following should be provided for this demo to run:
 - `VDB_PROVIDER`: the vector DB provider to be used. Must be supported by Llama Stack. For this demo, we use Milvus Lite which is our preferred solution.
 - `VDB_EMBEDDING`: the embedding model to be used for ingestion and retrieval. For this demo, we use all-MiniLM-L6-v2.
 - `VDB_EMBEDDING_DIMENSION` (optional): the dimension of the embedding. Defaults to 384.
 - `VECTOR_DB_CHUNK_SIZE` (optional): the chunk size for the vector DB. Defaults to 512.

## 1. Setting Up the Environment
We will start with a few imports needed for this demo only.

In [1]:
import uuid

from llama_stack_client import Agent, AgentEventLogger, RAGDocument
from llama_stack_client.lib.agents.event_logger import EventLogger

Next, we will initialize our environment as described in detail in our ["Getting Started" notebook](demos/rag_agentic/notebooks/Level0_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [2]:
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# pretty print of the results returned from the model/agent
import sys
sys.path.append('..')  
from src.utils import step_printer
from termcolor import cprint

remote = os.getenv("REMOTE", "True")

if remote == "False":
    local_port = os.getenv("LOCAL_SERVER_PORT", 8321)
    base_url = f"http://localhost:{local_port}"
else: # any value non equal to 'False' will be considered as 'True'
    base_url = os.getenv("REMOTE_BASE_URL")


# Tavily search API key is required for some of our demos and must be provided to the client upon initialization.
# We will cover it in the agentic demos that use the respective tool. Please ignore this parameter for all other demos.
tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY")
if tavily_search_api_key is None:
    provider_data = None
else:
    provider_data = {"tavily_search_api_key": tavily_search_api_key}


client = LlamaStackClient(
    base_url=base_url,
    provider_data=provider_data
)
    
print(f"Connected to Llama Stack server @ {base_url}")

# model_id will later be used to pass the name of the desired inference model to Llama Stack Agents/Inference APIs
model_id = os.getenv("INFERENCE_MODEL_ID")

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "True")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

Connected to Llama Stack server @ http://localhost:8321
Inference Parameters:
	Model: ibm-granite/granite-3.2-8b-instruct
	Sampling Parameters: {'strategy': {'type': 'greedy'}, 'max_tokens': 4096}
	stream: True


Finally, we will initialize the document collection to be used for RAG ingestion and retrieval.

In [20]:
vector_db_id = f"test_vector_db_{uuid.uuid4()}"

## 2. Indexing the Documents
- Initialize a new document collection in the target vector DB. All parameters related to the vector DB, such as the embedding model and dimension, must be specified here.
- Provide a list of document URLs to the RAG tool. Llama Stack will handle fetching, conversion and chunking of the documents' content.

In [21]:
# define and register the document collection to be used
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model=os.getenv("VDB_EMBEDDING"),
    embedding_dimension=int(os.getenv("VDB_EMBEDDING_DIMENSION", 384)),
    provider_id=os.getenv("VDB_PROVIDER"),
)

# ingest the documents into the newly created document collection
urls = [
    ("https://www.openshift.guide/openshift-guide-screen.pdf", "application/pdf"),
]
documents = [
    RAGDocument(
        document_id=f"num-{i}",
        content=url,
        mime_type=url_type,
        metadata={},
    )
    for i, (url, url_type) in enumerate(urls)
]
client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=int(os.getenv("VECTOR_DB_CHUNK_SIZE", 512)),
)

## 3. Executing queries via the RAG-aware agent
- Initialize an agent with a list of tools including the built-in RAG tool. The RAG tool specification must include a list of document collection IDs to retrieve from.
- For each prompt, initialize a new agent session, execute a turn during which a retrieval call may be requested, and output the reply received from the agent.

In [22]:
queries = [
    "How to install OpenShift?",
]

# initializing the agent
agent = Agent(
    client,
    model=model_id,
    instructions="You are a helpful assistant.",
    sampling_params=sampling_params,
    # we make our agent aware of the RAG tool by including builtin::rag/knowledge_search in the list of tools
    tools=[
        dict(
            name="builtin::rag/knowledge_search",
            args={
                "vector_db_ids": [vector_db_id],  # list of IDs of document collections to consider during retrieval
            },
        )
    ],
)

for prompt in queries:
    cprint(f"\nUser> {prompt}", "blue")
    
    # create a new turn with a new session ID for each prompt
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=agent.create_session(f"rag-session_{uuid.uuid4()}"),
        stream=stream,
    )
    
    # print the response, including tool calls output
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps)


User> How to install OpenShift?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: knowledge_search, Arguments: {'query': 'installing OpenShift'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


[
│   TextContentItem(
│   │   text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n',
│   │   type='text'
│   ),
│   TextContentItem(
│   │   text='Result 1:\nDocument_id:num-0\nContent:  We\nrecommend you to check the official Red Hat OpenShift Local documentation for an updated list of\nrequirements at the official documentation website.\n\uf05a\nRegarding Linux, even if Red Hat does not officially support them, OpenShift Local\ncan run on other distributions, such as Ubuntu or Debian, with minor caveats.\nRunning OpenShift Local on any Linux distribution requires a few additional\nsoftware packages to be installed through your default package manager. The\n15\ndocumentation at crc.dev/crc has more information about this subject.\n7.2. Hardware Requirements\nIn terms of hardware, OpenShift Local has some strict requirements. Your system must use a recent\nIntel CPU (except for Macs, where Apple Silicon machines are supported) with at least four physical\ncores and have at least 16 GB of RAM. Be aware that the base installation of OpenShift Local\nrequires at least 9 GB free to start. Of course, to run other applications on OpenShift Local, you will\nneed more RAM, so using a computer with at least 32 GB of RAM is strongly recommended.\nOpenShift Local also requires at least 35 GB of free disk space for its installation. The memory\nrequirements are likely to increase in the future, so please check the documentation at crc.dev for\nmore up-to-date information.\n7.3. Installation\nTo install OpenShift Local, open your web browser and navigate to console.redhat.com/openshift/\ncreate/local . Download the latest release of OpenShift Local and the "pull secret" file. The latter is a\nfile containing a key identifying your copy of OpenShift Local to your Red Hat Developer account.\nUnzip the file containing the OpenShift Local executable, and using your terminal, run the\ncommand crc setup . This command will prepare your copy of OpenShift Local, verifying\nrequirements and setting the required configuration values.\nOnce the crc setup command is ready, launch crc start. Running crc start can take a long time,\naround 20 minutes, on a recent PC.\nOnce started, access the OpenShift Web Console with the crc console command, which will open\nyour default browser. OpenShift Local uses the developer username and password to log in as a\nlow-privilege user, while the kubeadmin user uses a random-generated password. Use the crc\nconsole --credentials command to find the credentials required to log in as the kubeadmin user.\nOpenShift Local allows developers to perform various everyday tasks as if it were a standard\nOpenShift cluster, like deploying applications\n',
│   │   type='text'
│   ),
│   TextContentItem(
│   │   text='Result 2:\nDocument_id:num-0\nContent: .\nThese characteristics set OpenShift apart as an excellent Kubernetes platform for enterprise users.\nThe latest version of OpenShift available at the time of this writing is 4.12.\n3.2. Is Red Hat OpenShift Open Source?\nRed Hat OpenShift is a commercial product based on an open-source project called OKD. This\nacronym means " OpenShift Kubernetes Distribution" and is publicly available for everyone to\ninspect and contribute. Like the upstream Kubernetes project, OKD developers use the Go\nprogramming language.\n3.3. How can I run OpenShift?\nToday, Red Hat OpenShift is available through various mechanisms and formats:\n• DevOps teams can install it in their data centers "on-premise."\n• Major hyperscalers such as AWS, Azure, Google Cloud Platform, and IBM Cloud offer managed\nRed Hat OpenShift installations.\n• Developers can either run OpenShift locally on their workstations using Red Hat OpenShift\nLocal, also known as CRC or "Code-Ready Containers"\n• They can also request a 30-day trial OpenShift cluster, offered by Red Hat, at no charge, for\ntesting and evaluation purposes.\nRed Hat OpenShift is an integrated Platform-as-a-Service f


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Based on the provided text, here is an answer to the query:

To install Red Hat OpenShift, you can follow these steps:

1. Download and install the oc tool from the Red Hat website or through the "Help" menu on the OpenShift Web Console.
2. Install OpenShift Local by downloading a JAR file and running it with Java.
3. Use the oc tool to create a new project and import a Git repository, YAML file, or container image into your project.

Alternatively, you can use the odo tool provided by Red Hat to create applications using Devfiles. To do this:

1. Download the odo tool from the "Help" menu on the OpenShift Web Console.
2. Use the odo init command to prompt for a new application and select a programming language.
3. The odo push command builds and pushes the container to the OpenShift container registry, deploying it onto your cluster.

Note: These steps assume you have a Red Hat developer account and access to the OpenShi

## Key Takeaways
This tutorial demonstrates how to implement agentic RAG with Llama Stack. We do so by initializing an agent while giving it access to the RAG tool, then invoking the agent on each of the specified queries. Please check out our [complementary tutorial](demos/rag_agentic/notebooks/Level1_foundational_RAG.ipynb) for a non-agentic RAG example.